## Project Navigation - Report

### Training step

We construct the **agent** with different parameters at each steps of training.
Subsquently, we run the *Deep-Q-Network* procedure **dqn** as follows:

        agent = Agent(state_size=37, action_size=4, seed=1, fc1_units=fc1_nodes, fc2_units=fc2_nodes)       
        scores, episodes = **dqn**(n_episodes = 2000, eps_start = epsilon_start, train_numb=i)  
    
The file 'weights_'+str(train_numb)+'.trn' is  the obtained weights saved file.



### Deep-Q-Network algorithm

The order of _Deep-Q-Network_ **dqn** excutes the external loop (by _episodes_) till the number of episodes 
hits the max num of episodes _n_episodes = 2000_.
We check the below for looking up to the how many episodes are run 

        np.mean(scores_window) >= 13,  

where _scores_\__window_ is the array of the type deque realizing  the shifting window of length <= 100.
The _score_ is contained in the element _scores_\__window_[i] then it is achieved by the algorithm on the episode _i_.


internaly,  **dqn** gets the current _action_ from the **agent**.
By this _action_ **dqn** gets _state_ and _reward_ from Unity environment _env_.
Then, the **agent** accept params _state_, _action_, _reward_, _next_\__state_, _done_
At the following training step. The _score_ accumulates attained rewards.


### Mechanisms of Agent

The class **Agent** is is the well-known class implementing the following mechanisms:

* Two Q-Networks (local and target) using the simple neural network.

        self.qnetwork_local = QNetwork(state_size, action_size, seed, fc1_units, fc2_units).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed, fc1_units, fc2_units).to(device)

* Replay memory (using the class ReplayBuffer)

       self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
       ...
       e = self.experience(state, action, reward, next_state, done)
       self.memory.append(e)
     
* Epsilon-greedy mechanism

        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
            
   The epsilon become a bit smaller with each episode:
   
        eps = max(eps_end, eps_decay*eps), 
        
where eps_end=0.01, eps_decay = 0.996.        
   
* Q-learning, i.e., using the max value for all possible actions
* Computing the loss function by MSE loss

       loss = F.mse_loss(Q_expected, Q_targets)
     
* Minimize the loss by gradient descend mechanism using the ADAM optimizer

### Model Q-Network

Both Q-Networks (local and target) are implemented by the class
**QNetwork**. This class implements the simple neural network    
with 3 fully-connected layers and 2 rectified nonlinear layers.
The class **QNetwork** is realized in the framework of package **PyTorch**.   
The number of neurons of the fully-connected layers are as follows:

 * Layer fc1,  number of neurons: _state_\__size_ x _fc1_\__units_, 
 * Layer fc2,  number of neurons: _fc1_\__units_ x _fc2_\__units_,
 * Layer fc3,  number of neurons: _fc2_\__units_ x _action_\__size_,
 
where _state_\__size_ = 37, _action_\__size_ = 8, _fc1_\__units_ and _fc2_\__units_
are the input params.

### Training and Testing 
 
We run 5 training sessions with different parameters _fc1_\__units_,  _fc2_\__units_, _eps_\__start_,
and we save obtained weights by the function of **PyTorch**:

    torch.save(agent.qnetwork_local.state_dict(), 'weights_'+str(train_numb)+'.trn') 
     
For input: fc1_units = 45, fc2_units = 41, we get the following training output:   

train_numb:  0 eps_start:  0.975
Episode: 538, elapsed: 0:10:51.311591, Avg.Score: 13.02,  score 19.0, How many scores >= 13: 59, eps.: 0.11
 terminating at episode : 538 ave reward reached +13 over 100 episodes 

![](layers_48x40.png)


After that we go to the testing session. For each session, we load saved weights by the function of **PyTorch** 
as follows:

    file_weights = 'weights_'+str(train_n)+'.trn'
    agent.qnetwork_local.load_state_dict(torch.load(file_weights))

### Solved environment (from my local machine)

We get the average score > 13 in 5 from 6 testing sessions.  The number of episodes needed to reach this score 
less than 600. Only in the case fc1_units = 96, fc2_units = 96 we get the average score < 13, however in this case
in 3 tests from 6 we get the average score > 13.  From the examples that I have seen so far (on my local machine and on
the workspace machine), we get better results when fc1_inits > fc2_units. 